# Họ tên: Nguyễn Vũ Hải
# Lớp: CNTT K18 CLC
# Mã SV: DTC195480201CLC0006

In [3]:
import sys

In [15]:
if 'pyspark' not in sys.modules.keys():
    !pip install pyspark
else:
    print("Đã cài pyspark")

In [16]:
from pyspark.sql import SparkSession

# 1.  Đọc file products.csv trên vào 1 dataframe có tên product_df.

a. Hiển thị schema, 10 dòng đầu và 10 dòng cuối của product_df

b. Hiển thị thống kê của product_df

c. Hiển thị số lượng đơn hàng của mỗi khách hàng theo tên trong customer_df

d. Hãy gộp 2 dataframe product_df và order_data thành dataframe có tên order_product với các thông tin customer_id, product_id, price, delivery_country.

e. Hãy gộp 2 dataframe order_product và customer_data với các thông tin name, id, product_id, price, billing_country, delivery_country.

f. Hãy gộp 2 dataframe order_product và customer_data với các thông tin name, id, product_id, price, delivery_country khi giao hàng trong nước (delivery_country = billing_country)

In [17]:
# Create a SparkSession
spark = SparkSession.builder.appName("Analysis").getOrCreate()
spark

# a,b

In [48]:
# Load the data from a file into a DataFrame
product_df = spark.read.format('csv') \
                .option("header", "true") \
                .load("products.csv")
product_df = product_df.withColumnRenamed("id", "id_product")

# In dataframe
print(product_df)

# Hiển thị loại dữ liệu từng trường thông tin
print("Dtypes: " % product_df.dtypes)
 
print("Schema: ")
product_df.printSchema()
print("Hiển thị 5 dòng đầu tiên:")
display(product_df.head(5))

print("Hiển thị 5 dòng cuối cùng:")
display(product_df.tail(5))


print("Hiển thị mô tả thống kê: ")
product_df.describe().show()

DataFrame[id_product: string, price: string]
Dtypes: 
Schema: 
root
 |-- id_product: string (nullable = true)
 |-- price: string (nullable = true)

Hiển thị 5 dòng đầu tiên:


[Row(id_product='0', price='21.78'),
 Row(id_product='1', price='18.48'),
 Row(id_product='2', price='24.17'),
 Row(id_product='3', price='29.71'),
 Row(id_product='4', price='20.12')]

Hiển thị 5 dòng cuối cùng:


[Row(id_product='995', price='9.65'),
 Row(id_product='996', price='25.96'),
 Row(id_product='997', price='13.33'),
 Row(id_product='998', price='28.02'),
 Row(id_product='999', price='18.86')]

Hiển thị mô tả thống kê: 
+-------+-----------------+-----------------+
|summary|       id_product|            price|
+-------+-----------------+-----------------+
|  count|             1000|             1000|
|   mean|            499.5|19.98996999999997|
| stddev|288.8194360957494|5.999346335689877|
|    min|                0|            10.07|
|    max|              999|             9.97|
+-------+-----------------+-----------------+



# c

In [49]:
# Load the data from a file into a DataFrame
orders_df = spark.read.format('csv') \
                .option("header", "true") \
                .load("orders.csv")


# In dataframe
print(orders_df)

# Hiển thị loại dữ liệu từng trường thông tin
print("Dtypes: " % orders_df.dtypes)
 
print("Schema: ")
orders_df.printSchema()


DataFrame[id: string, customer_id: string, product_id: string, delivery_country: string]
Dtypes: 
Schema: 
root
 |-- id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- delivery_country: string (nullable = true)



In [50]:
# Load the data from a file into a DataFrame
customer_df = spark.read.format('csv') \
                .option("header", "true") \
                .load("customers.csv")
customer_df = customer_df.withColumnRenamed("id", "id_customer")
# In dataframe
print(customer_df)

# Hiển thị loại dữ liệu từng trường thông tin
print("Dtypes: " % customer_df.dtypes)
 
print("Schema: ")
customer_df.printSchema()
     

DataFrame[id_customer: string, name: string, billing_country: string]
Dtypes: 
Schema: 
root
 |-- id_customer: string (nullable = true)
 |-- name: string (nullable = true)
 |-- billing_country: string (nullable = true)



In [61]:
joined_df = orders_df.join(customer_df, orders_df["customer_id"] == customer_df["id_customer"]).join(product_df, orders_df["product_id"] == product_df["id_product"])
selected_df = joined_df.select("id", "name")
selected_df.groupBy("name").count().show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|       Felix Rantzen|    3|
|       Kazuhiro Nijo|    3|
|  Flomo Gebremichael|    1|
|      Hanna Jacobsen|    6|
|      Cumhur Ozdemir|   10|
|        Lukas Juhasz|    5|
|       Latoya Omondi|   13|
|Eber Castano Fern...|    3|
|       Chris  Wesley|    2|
|   Jean-Marc Chambon|    5|
|    Harumi Tokudaiji|    3|
|      Onofredo Tasso|    4|
|     Fang Ching-hsia|    5|
|       Jumanah Karim|    3|
|    Sofia Vinogradov|    2|
|           Sui Tso-i|    9|
|    Veeti Gustavsson|    9|
|     Catulle Mazarin|   10|
|        Momolu Okeke|    4|
|          Oso Jalloh|    5|
+--------------------+-----+
only showing top 20 rows



d. Hãy gộp 2 dataframe product_df và order_data thành dataframe có tên order_product với các thông tin customer_id, product_id, price, delivery_country.

e. Hãy gộp 2 dataframe order_product và customer_data với các thông tin name, id, product_id, price, billing_country, delivery_country.

f. Hãy gộp 2 dataframe order_product và customer_data với các thông tin name, id, product_id, price, delivery_country khi giao hàng trong nước (delivery_country = billing_country)


# d

In [64]:
order_product = orders_df.join(product_df, orders_df["product_id"] == product_df["id_product"])
print(order_product)
order_product.select("customer_id","product_id","price","delivery_country").show()

DataFrame[id: string, customer_id: string, product_id: string, delivery_country: string, id_product: string, price: string]
+-----------+----------+-----+----------------+
|customer_id|product_id|price|delivery_country|
+-----------+----------+-----+----------------+
|       1866|       686|26.62|           India|
|       1921|       617| 8.61|           China|
|       1921|        54| 35.6|         Armenia|
|       1921|       566|23.32|           China|
|       1921|       128|13.73|         Lebanon|
|       1921|       976|29.55|           China|
|       1907|       353|16.81|           Spain|
|       1464|       330|14.39|         Denmark|
|       1464|       555|19.12|         Denmark|
|        206|       960|19.96|          Mexico|
|        206|       630|15.47|          Mexico|
|        206|        13|14.37|          Mexico|
|       1496|       179|21.09|         Finland|
|       1496|       927|  7.9|         Burundi|
|       1496|       873| 23.5|         Finland|
|       1496

# e

In [73]:
order_customer = orders_df.join(customer_df, orders_df["customer_id"] == customer_df["id_customer"]).join(product_df, orders_df["product_id"] == product_df["id_product"])
print(order_customer)
order_customer.select("id","name", "product_id","price", "billing_country","delivery_country").show()

DataFrame[id: string, customer_id: string, product_id: string, delivery_country: string, id_customer: string, name: string, billing_country: string, id_product: string, price: string]
+---+-----------------+----------+-----+---------------+----------------+
| id|             name|product_id|price|billing_country|delivery_country|
+---+-----------------+----------+-----+---------------+----------------+
|  0|Anangam Ramamuthe|       686|26.62|          India|           India|
|  1| Tsung Xiang-ling|       617| 8.61|          China|           China|
|  1| Tsung Xiang-ling|        54| 35.6|          China|         Armenia|
|  1| Tsung Xiang-ling|       566|23.32|          China|           China|
|  1| Tsung Xiang-ling|       128|13.73|          China|         Lebanon|
|  1| Tsung Xiang-ling|       976|29.55|          China|           China|
|  2|    Chao Wu-jiang|       353|16.81|          China|           Spain|
|  3|  Arttu Pettersen|       330|14.39|        Denmark|         Denmark|
| 

# f. Hãy gộp 2 dataframe order_product và customer_data với các thông tin name, id, product_id, price, delivery_country khi giao hàng trong nước (delivery_country = billing_country)

In [80]:
order_domestic = orders_df.join(customer_df, (orders_df["customer_id"] == customer_df["id_customer"]) & (orders_df["delivery_country"] == customer_df["billing_country"])).join(product_df, orders_df["product_id"] == product_df["id_product"])
print(order_customer)
order_domestic.select("id","name", "product_id","price","delivery_country").show()

DataFrame[id: string, customer_id: string, product_id: string, delivery_country: string, id_customer: string, name: string, billing_country: string, id_product: string, price: string]
+---+-----------------+----------+-----+----------------+
| id|             name|product_id|price|delivery_country|
+---+-----------------+----------+-----+----------------+
|  0|Anangam Ramamuthe|       686|26.62|           India|
|  1| Tsung Xiang-ling|       617| 8.61|           China|
|  1| Tsung Xiang-ling|       566|23.32|           China|
|  1| Tsung Xiang-ling|       976|29.55|           China|
|  3|  Arttu Pettersen|       330|14.39|         Denmark|
|  3|  Arttu Pettersen|       555|19.12|         Denmark|
|  4| Aura Santa Godoy|       960|19.96|          Mexico|
|  4| Aura Santa Godoy|       630|15.47|          Mexico|
|  4| Aura Santa Godoy|        13|14.37|          Mexico|
|  5|   Tobias Eriksen|       179|21.09|         Finland|
|  5|   Tobias Eriksen|       873| 23.5|         Finland|
|  5

# Bài 2. Tạo 1 ứng dụng Spark tên "practice_02" để đọc tệp "Salaries.csv" vào Pyspark DataFrame.

a)Xóa các cột không có thông tin trong df

b) Hiển thị thống kê của df

c) Xóa các dòng có tiền lương tổng (TotalPay) <=0

d) Điền giá trị 0 vào các ô thiếu dữ liệu.

e) Hiển thị tên và nghề nghiệp những người có tổng lương < 50000

In [90]:
# Load the data from a file into a DataFrame
Salaries_df = spark.read.format('csv') \
                .option("header", "true") \
                .load("Salaries.csv")
# In dataframe
print(Salaries_df)

# Hiển thị loại dữ liệu từng trường thông tin
print("Dtypes: " % Salaries_df.dtypes)
 
print("Schema: ")
Salaries_df.printSchema()

Salaries_df.head(5)

DataFrame[Id: string, EmployeeName: string, JobTitle: string, BasePay: string, OvertimePay: string, OtherPay: string, Benefits: string, TotalPay: string, TotalPayBenefits: string, Year: string, Notes: string, Agency: string, Status: string]
Dtypes: 
Schema: 
root
 |-- Id: string (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- BasePay: string (nullable = true)
 |-- OvertimePay: string (nullable = true)
 |-- OtherPay: string (nullable = true)
 |-- Benefits: string (nullable = true)
 |-- TotalPay: string (nullable = true)
 |-- TotalPayBenefits: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Status: string (nullable = true)



[Row(Id='1', EmployeeName='NATHANIEL FORD', JobTitle='GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY', BasePay='167411.18', OvertimePay='0.0', OtherPay='400184.25', Benefits=None, TotalPay='567595.43', TotalPayBenefits='567595.43', Year='2011', Notes=None, Agency='San Francisco', Status=None),
 Row(Id='2', EmployeeName='GARY JIMENEZ', JobTitle='CAPTAIN III (POLICE DEPARTMENT)', BasePay='155966.02', OvertimePay='245131.88', OtherPay='137811.38', Benefits=None, TotalPay='538909.28', TotalPayBenefits='538909.28', Year='2011', Notes=None, Agency='San Francisco', Status=None),
 Row(Id='3', EmployeeName='ALBERT PARDINI', JobTitle='CAPTAIN III (POLICE DEPARTMENT)', BasePay='212739.13', OvertimePay='106088.18', OtherPay='16452.6', Benefits=None, TotalPay='335279.91', TotalPayBenefits='335279.91', Year='2011', Notes=None, Agency='San Francisco', Status=None),
 Row(Id='4', EmployeeName='CHRISTOPHER CHONG', JobTitle='WIRE ROPE CABLE MAINTENANCE MECHANIC', BasePay='77916.0', OvertimePay='56120.71'

In [92]:
Salaries_df.dropna(thresh=2).show()

+---+------------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+
| Id|      EmployeeName|            JobTitle|  BasePay|OvertimePay| OtherPay|Benefits| TotalPay|TotalPayBenefits|Year|Notes|       Agency|Status|
+---+------------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+
|  1|    NATHANIEL FORD|GENERAL MANAGER-M...|167411.18|        0.0|400184.25|    null|567595.43|       567595.43|2011| null|San Francisco|  null|
|  2|      GARY JIMENEZ|CAPTAIN III (POLI...|155966.02|  245131.88|137811.38|    null|538909.28|       538909.28|2011| null|San Francisco|  null|
|  3|    ALBERT PARDINI|CAPTAIN III (POLI...|212739.13|  106088.18|  16452.6|    null|335279.91|       335279.91|2011| null|San Francisco|  null|
|  4| CHRISTOPHER CHONG|WIRE ROPE CABLE M...|  77916.0|   56120.71| 198306.9|    null|332343.61|       332343.61|2011| null|